In [3]:
#Example of loading image automatically
#https://picsum.photos/ 
from PIL import Image
import requests
im = Image.open(requests.get('https://picsum.photos/512/512', stream=True).raw)

In [5]:
im.save('imm.jpg')

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
#https://gist.github.com/wshanshan/c825efca4501a491447056849dd207d6
!pip install colour

In [21]:
from PIL import Image, ImageDraw, ImageFont
from colour import Color
import numpy as np

In [22]:
def asciiart(in_f, SC, GCF,  out_f, color1='black', color2='blue', bgcolor='white'):

    # The array of ascii symbols from white to black
    chars = np.asarray(list(' .,:irs?@9B&#'))

    # Load the fonts and then get the the height and width of a typical symbol 
    # You can use different fonts here
    font = ImageFont.load_default()
    letter_width = font.getsize("x")[0]
    letter_height = font.getsize("x")[1]

    WCF = letter_height/letter_width

    #open the input file
    #img = Image.open(in_f)
    img = in_f

    widthByLetter=round(img.size[0]*SC*WCF)
    heightByLetter = round(img.size[1]*SC)
    S = (widthByLetter, heightByLetter)

    #Resize the image based on the symbol width and height
    img = img.resize(S)
    
    img = np.sum(np.asarray(img), axis=2)
    
    # Normalize the results, enhance and reduce the brightness contrast. 
    # Map grayscale values to bins of symbols
    img -= img.min()
    img = (1.0 - img/img.max())**GCF*(chars.size-1)
    
    # Generate the ascii art symbols 
    lines = ("\n".join( ("".join(r) for r in chars[img.astype(int)]) )).split("\n")

    # Create gradient color bins
    nbins = len(lines)
    colorRange =list(Color(color1).range_to(Color(color2), nbins))

    #Create an image object, set its width and height
    newImg_width= letter_width *widthByLetter
    newImg_height = letter_height * heightByLetter
    newImg = Image.new("RGBA", (newImg_width, newImg_height), bgcolor)
    draw = ImageDraw.Draw(newImg)

    # Print symbols to image
    leftpadding=0
    y = 0
    lineIdx=0
    for line in lines:
        color = colorRange[lineIdx]
        lineIdx +=1

        draw.text((leftpadding, y), line, color.hex, font=font)
        y += letter_height

    # Save the image file

    #out_f = out_f.resize((1280,720))
    newImg.save(out_f)

In [23]:
def imgGen(img1,count):
  inputf = img1  # Input image file name

  SC = 0.1    # pixel sampling rate in width
  GCF= 2      # contrast adjustment

  asciiart(inputf, SC, GCF, "results.png")   #default color, black to blue
  asciiart(inputf, SC, GCF, "results_pink.png","blue","pink")
  img = img1
  img2 = Image.open('results.png').resize(img.size)
  img2.save('result.png')
  img3 = Image.open('results_pink.png').resize(img.size)
  img3.save('resultp.png')

  images = [img2,img]#change
  widths, heights = zip(*(i.size for i in images))

  total_width = sum(widths)
  max_height = max(heights)

  new_im = Image.new('RGB', (total_width, max_height))

  x_offset = 0
  for im in images:
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]

  img4 = new_im.resize((1024,512))
  img4.save('drive/MyDrive/trainingAsciiGan/w11'+str(count)+'.jpg')

  images = [img3,img]#change
  widths, heights = zip(*(i.size for i in images))

  total_width = sum(widths)
  max_height = max(heights)

  new_im = Image.new('RGB', (total_width, max_height))

  x_offset = 0
  for im in images:
    new_im.paste(im, (x_offset,0))
    x_offset += im.size[0]

  img5 = new_im.resize((1024,512))
  img5.save('drive/MyDrive/trainingAsciiGan/w12'+str(count+1)+'.jpg')

In [24]:
import os
count = 0
while count < 200:
  im = Image.open(requests.get('https://picsum.photos/512/512', stream=True).raw)
  imgGen(im,count)
  count += 1